# Live training | 2023-06-13 | Building AI Applications with LangChain and GPT

You've probably talked to ChatGPT using the web interface, or used the API with the `openai` python package and wondered "what if I could teach it about my own data?". Today we're going to build such an application using LangChain, a framework for developing applications powered by language models.

In today's session, we'll build a chatbot powered by GPT-3.5 that can answer questions about LangChain, as it will have knowledge of the LangChain documentation. We'll cover:
- Getting setup with an OpenAI developer account and integration with Workspace;
- Install the LangChain package
- Preparing the data
- Embed the data using OpenAI's Embed API, and get a cost estimate for this operation
- Storing the data in a vector database
- How to query the vector database
- Putting together a basic chat application to "talk to the LangChain docs"

## Before you begin

### Create a developer account with OpenAI

1. Go to the [API signup page](https://platform.openai.com/signup). 

2. Create your account (you'll need to provide your email address and your phone number).

<img src="images/openai-create-account.jpeg" width="200">

3. Go to the [API keys page](https://platform.openai.com/account/api-keys). 

4. Create a new secret key.

<img src="images/openai-new-secret-key.png" width="200">

5. **Take a copy of it**. (If you lose it, delete the key and create a new one.)

### Add a payment method

OpenAI sometimes provides free credits for the API, but it's not clear if that is worldwide or what the conditions are. You may need to add debit/credit card details. 

We will use 2 APIs:
- The Chat API with the `gpt-3.5-turbo` model (cost: $0.002 / 1K tokens)
- The Embedding API with the `Ada v2` model (cost: $0.0004 / 1K tokens)

**In total, the Chat API (used for completions) should cost less than `$0.1` and embedding should cost around `$0.6`. This notebook provides embeddings already, so you can skip the embedding step.**

1. Go to the [Payment Methods page](https://platform.openai.com/account/billing/payment-methods).

2. Click Add payment method.

<img src="images/openai-add-payment-method.png" width="200">

3. Fill in your card details.

### Set up Environment Variables

1. In Workspace, click on Environment.

<img src="images/workspace-environment.png" width="200">

2. Click on the "Environment Variables" plus button.

<img src="images/workspace-new-env-vars.png" width="200">

3. In the "Name" field, type `OPENAI_API_KEY`. In the "Value" field, paste in your secret key (starting with `sk-`)

<img src="images/workspace-create-env-vars.png" width="300">

4. Click "Create", and connect the new integration.

<img src="images/workspace-connect-env-vars.png" width="300">


## Task 0: Setup

For the purpose of this training, we'll need to install a few packages:
- [`langchain`](https://python.langchain.com/en/latest/index.html): The LangChain framework, has many releases and must use a stable one
- [`chromadb`](https://docs.trychroma.com/): The package we'll use for the vector database
- [`tiktoken`](https://github.com/openai/tiktoken): A tokenizer we'll use to count GPT-3 tokens

In [1]:
# install openai (version 0.27.1)
!pip install openai==0.27.1
# install langchain (version 0.0.191)
!pip install langchain==0.0.191
# install chromadb
!pip install chromadb==0.3.26
# install tiktoken
!pip install tiktoken==0.4.0

## Task 1: Load data

To be able to embed and store data, we need to provide LangChain with Documents. This is easy to achieve in LangChain thanks to [Document Loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html). In our case, we're targeting a "Read the docs" documentation, for which there is a loader [`ReadTheDocsLoader`](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/readthedocs_documentation.html).
In the folder `rtdocs`, you'll find all the HTML files from the LangChain documentation (https://python.langchain.com/en/latest/index.html). 

<details>
    <summary>How did we obtain the data</summary>
<br/><p>These file were downloaded by executing this linux command:</p>
<pre>
wget -r -A.html -P rtdocs https://python.langchain.com/en/latest/
</pre>
<p>We urge you **NOT** to execute this during the live training, as it will scan and download the full langchain doc site (~1000 files). This operation may be heavy and could disrupt the site, especially if hundreds of learners do it all at once!</p>
</details>
<br/><br/>


Our first task is to load these HTML files as documents that we can use with langchain: we're going to use the `ReadTheDocsLoader`. It will read the directory containing all HTML files and transform them into `Document` objects. `ReadTheDocsLoader` will read each HTML file, remove HTML tags to only keep the text and return it as a `Document`. At the end of this task, we'll have a variable `raw_documents` containing a list of `Document`: one `Document` per HTML file. 

Note that in this step we won't actually load the documents into a database, we're simply loading the documents in a list.

### Instructions
1. import `ReadTheDocsLoader` from `langchain.document_loaders`
2. Create the loader, pointing to the `rtdocs/python.langchain.com/en/latest` directory and enabling the HTML parser feature with `features='html.parser'`
3. Load the data in `raw_documents` by calling `loader.load()`


In [2]:
# Import ReadTheDocsLoader
from langchain.document_loaders import ReadTheDocsLoader

# Create a loader for the `rtdocs/python.langchain.com/en/latest` folder
loader = ReadTheDocsLoader("rtdocs/python.langchain.com/en/latest", features='html.parser')

# Load the data
raw_documents = loader.load()

## Task 2: Slice the documents into smaller chunks

In the previous step, we turned each HTML file into a Document. These files may be very long, and are potentially too large to embed fully. It's also a good practice to avoid embedding large documents:
- long documents often contain several concepts. Retrieval will be easier if each concept is indexed separately;
- retrieved documents will be injected in a prompt, so keeping them short will keep the prompt small(ish)

LangChain has a collection of tools to do this: [Text Splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html). In our case, we'll be using the most straightfoward one and simplest to use: the [Recursive Character Text Splitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html). The recursive text splitter will recursively reduce the input by splitting it by paragraph, then sentences, then words as needed until the chunk is small enough. 

### Instructions
1. Import the `RecursiveCharacterTextSplitter` from `langchain.text_splitter`
2. Create a text splitter configured with `chunk_size=1000` and `chunk_overlap=200`  
   _These values are arbitrary and you'll need to try different ones to see which best serve your use case_
3. split the `raw_documents` and store them as `documents`, using the `.split_documents()` method

In [3]:
# Import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # arbitrary, experience with your own data is important
    chunk_overlap=200, # choose as well, we have overlap to retain context
)

# Split the documents
documents = text_splitter.split_documents(raw_documents)

## Task 3: count tokens and get a cost estimate of embedding

We're now ready to embed our documents. Before we do so, we'd like to get an idea of how big it is and how much it will cost to embed. To do so, we'll use the [`tiktoken`](https://github.com/openai/tiktoken) library (no relation to TikTok, there is no dancing involved). tiktoken allows to encode and decode strings of text into tokens. In our case, we're mostly interested in how many tokens our documents translate to.

> 💡 To better understand what a token is to GPT, head to [OpenAI's Tokenizer page](https://platform.openai.com/tokenizer) where you can see how a text translates to tokens.

Prices for different models can be found on their [pricing page](https://openai.com/pricing).

### Instructions
1. Import `tiktoken`
2. Create a tokenizer for the `text-embedding-ada-002` model using the `.encoding_for_model()` method
3. Count tokens in each document using the `.encode()` method
4. Calculate the sum of all tokens
5. Calculate a cost estimate. The `text-embedding-ada-002` model costs `$0.0004` for 1000 tokens

In [3]:
# Import tiktoken
import tiktoken

# Create an encoder 
encoder = tiktoken.encoding_for_model('text-embedding-ada-002')  # model used for the embedding

# Count tokens in each document
doc_tokens = [len(encoder.encode(d.page_content)) for d in documents]

# Calculate the sum of all token counts
total_tokens = sum(doc_tokens)

# Calculate a cost estimate
cost_estimate = (sum(doc_tokens)/1000) * 0.0004
print(f"{total_tokens} tokens, cost estimate: ${cost_estimate:.2f}")

## Task 4: embed the documents and store embeddings in the vector database

We're now ready to embed our documents. Since embedding costs money, we'll want to save the embeddings into a database. LangChain can take care of all that using a [Vector Store](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html).

There are plenty of vector stores to choose from (see the [full list](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)). Today we'll use [Chroma](https://docs.trychroma.com/), but you could be using any other as they have the same interface in LangChain. Once again you'll need to try many of them to see which best fits your use case: some vector stores have specific features (like multimodality or multilingual), so be sure to check them out.

Chroma is simple to use and can be persisted to disk. If you do not whish to embed the full set of documents yourself, feel free to skip this step and use the provided folder `chroma-data-langchain-docs`: we've already embedded all documents and persisted it in this folder.

### Instructions
1. Import `Chroma` from `langchain.vectorstores`
2. Import `OpenAIEmbeddings` from `langchain.embeddings.openai` 
3. Create the embedding function
4. Create a database from our documents, using `Chroma.from_documents()`. Pass the documents, embedding function and `persist_directory`.  
   **Warning: executing this will embed thousands of documents and will cost about $0.6**
5. Persist the data to disk by calling `.persist()` on the database

In [4]:
# Import chroma
from langchain.vectorstores import Chroma

# Import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# Create the mebedding function
embedding = OpenAIEmbeddings()

# Create a database from the documents and embedding function
#db = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory="my-embeddings")

# Persist the data to disk
#db.persist()

## Alternative: use the provided embeddings

We have already executed the step above to embed all documents and stored the result in the `chroma-data-langchain-docs` folder. Instead of embedding all the documents yourself, you can use these embeddings at no cost.

The result of this step is the same as the step above, but will not call the OpenAI API and cost nothing.

### Instructions
1. Import `Chroma` from `langchain.vectorstores`
2. Import `OpenAIEmbeddings` from `langchain.embeddings.openai`
3. Create the embedding function
4. Load the database from the `chroma-data-langchain-docs` directory, and provide the embedding function

In [2]:
# Import chroma
from langchain.vectorstores import Chroma

# Import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# Create the embedding function
embedding = OpenAIEmbeddings()

# Load the database from existing embeddings
db = Chroma(persist_directory="chroma-data-langchain-docs", embedding_function=embedding)

## Step 5: query the vector database

Now that we have a vector database, we can query it. A vector database stores embeddings (vectors) and allow to search through them using K-Nearest Neighbors algorithm (or a variation of it). When we query it the following will happen:
1. Embed the text query to obtain a vector. It is crucial that this embedding is made using the same embedding technique that was used to embed the documents;
2. Calculate the distance (or similarity) between the query vector and all other vectors;
3. Sort results by similarity;
4. Return the most similar documents.

To do this with LangChain, we can use the `.similarity_search_with_score()` method of the database.

### Instructions
1. Call the `similarity_search_with_score` on `db` with the search query as parameter. Store the results in `results`;
2. Print the results


In [3]:
# Call the `similarity_search_with_score` method on `db`
results = db.similarity_search_with_score("how to load data from wikipedia?")

# Print the results
for (doc, score) in results:
    print(f"Score: {score}\n{doc.page_content}\n----")

## Step 6: Create a QA chain

Let's put it all together into a chat-like application. We want the user to ask a question, then search for relevant documents. We'll then create a prompt that includes the documents and the question so GPT can answer it (if possible).

First, we'll query the database in a similar manner to previous step. We'll use `.similarity_search()`:

```python
question = "show an example of adding memory to a chain"
context_docs = db.similarity_search(question)
```

Next, we will create a prompt that contains the question and the relevant documents:

> You can think of a PromptTemplate as an fstring in python: values in curly brances are used as placeholder and will be replaced by values we pass when running the chain.

```python
prompt = PromptTemplate(
    template=""""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

Question: {question}
Helpful Answer:""",
    input_variables=["context", "question"]
)
```

To call the LLM with this prompt, we need to create an `LLMChain` and pass it an LLM and the prompt:

```python
llm = ChatOpenAI(temperature=0)
qa_chain = LLMChain(llm=llm, prompt=prompt)
```

We can now call our chain like so:

```python
qa_chain({"context": "<the context>", "question": "<the question>"})
```

This will return a dict with a `text` key containing the LLM response.

### Instructions

1. Import the necessary pieces:  
   ```python
   from langchain.prompts import PromptTemplate
   from langchain.chains.llm import LLMChain
   from langchain.chat_models import ChatOpenAI
   ```
2. Store the question as `question`, query the database and store the result as `context_docs`
3. Create a prompt with 2 variables: `context` and `question`
4. Create a chain with an LLM and the prompt
5. Call the chain and print the result. The LLM output is in the `text` key


In [15]:
# Import 
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI

# Set the question variable
question = "how can I make a chain remember previous messages?"


# Query the database as store the results as `context_docs`
context_docs = db.similarity_search(question)

# Create a prompt with 2 variables: `context` and `question`
prompt = PromptTemplate(
    template=""""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

Question: {question}
Helpful Answer:""",
    input_variables=["context", "question"]
)

# Create an LLM with ChatOpenAI
llm = ChatOpenAI(temperature=0)

# Create the chain
qa_chain = LLMChain(llm=llm, prompt=prompt)

# Call the chain
result = qa_chain({"context": "\n".join([d.page_content for d in context_docs]), "question": question})

# Print the result
print(result["text"])



## To go further

Our little chat app is working, but can be improved. Consider the following improvements:
- **Clean up the documents**.  
  Each document currently starts with useless text, and ends with a copyright notice. These texts do not provide value in our case and should be removed before embedding. Try to make an additional step after loading the `raw_documents`. In this step, iterate over all documents and find a way to remove unnecessary text.
- **Add streaming to the LLM**.  
  Instead of waiting for the full response, you can get a better experience by streaming the LLM response (much like the ChatGPT web interface). Look at the `playground.ipynb` notebook for an example, or check the [docs](https://python.langchain.com/en/latest/modules/models/chat/examples/streaming.html).
- **Return sources to the user**.  
  Getting a response is nice, but linking to the relevant docs is even better. There are a few techniques to return the source documents to the user. The simplest is to print the metadata of all the `context_docs` returned by the semantic search. In this example, you could use `doc.metadata["source"]` to create a link to the langchain docs. A more advanced technique can be found [here](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa_with_sources.html).
- **Add memory support to the chain**.  
  You can pass a `memory` argument to the `LLMChain`. Since we have multiple input variables, make sure to specify which one the memory should use: `ConversationBufferMemory(input_key="question")`. You will also need to rerite the question before searching for documents, as the full context will not always be contained in the user input. You can find a working example in `going-further/adding-memory.ipynb`

<br />

### Useful links
- Learn more about embeddings: https://txt.cohere.com/sentence-word-embeddings/
- Learn more about vector databases: https://www.pinecone.io/learn/vector-database/


## LangChain caveats

While LangChain is a great tool to discover concepts and techniques, it falls short to help you deliver a production-ready application:
- Documentation is lacking. The documentation is not very complete and mostly made of examples, it lacks explanations and proper descriptions. To fully discover how a given module is working, you will need to peek in the code ([here on GitHub](https://github.com/hwchase17/langchain))
- Too many abstractions. LangChain brings a lot of classes, some would argue too many. A `PromptTemplate` for example is pretty much the equivalent of a simple python f-string.
- Very opinionated. As soon as you want to customize a chain (or agent), you will find that it's not always the smost simple thing. Many end-up re-implementing LangChain classes or functions to suit their needs better.

LangChain is a great starting point and will let you write compelling demos very quickly. Consider these as prototypes, and not something that can be brought to production.